In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

In [81]:
TEST_PATH = 'data/test.csv'
SUBMIT_PATH = 'data/submission.csv'

In [44]:
submit = pd.read_csv(SUBMIT_PATH)
submit.head()

,ego_id,u,v,x1
0,8,0,93,0.000000
1,8,0,143,0.000000
2,8,0,151,1.606742
3,8,1,24,0.026496
4,8,5,4,0.159857


In [17]:
submit.shape

(810976, 1)

In [15]:
test = pd.read_csv(
    TEST_PATH, 
    dtype={
        'u': np.int16, 
        'v': np.int16, 
        't': np.float32, 
        'x1': np.float32, 
        'x2': np.float32, 
        'x3': np.float32
    }
)
test.head()

,ego_id,u,v,t,x1,x2,x3
0,8,20,19,185.699997,3.839089e-04,0.0,0.0
1,8,131,125,161.399994,4.034464e-01,0.0,0.0
2,8,73,56,127.000000,8.554643e-05,0.0,0.0
3,8,0,4,594.500000,2.886418e-01,0.0,0.0
4,8,63,73,127.000000,4.281692e-07,0.0,0.0


In [18]:
test.shape

(40548780, 4)

In [14]:
submit.set_index(['ego_id', 'u', 'v'], inplace=True)
submit.head()

x1
ego_id u v            
8      0 93   0.000000
         143  0.000000
         151  1.606742
       1 24   0.026496
       5 4    0.159857

In [16]:
test.set_index(['ego_id', 'u', 'v'], inplace=True)
test.head()

t            x1   x2   x3
ego_id u   v                                      
8      20  19   185.699997  3.839089e-04  0.0  0.0
       131 125  161.399994  4.034464e-01  0.0  0.0
       73  56   127.000000  8.554643e-05  0.0  0.0
       0   4    594.500000  2.886418e-01  0.0  0.0
       63  73   127.000000  4.281692e-07  0.0  0.0

In [21]:
new_submit = test.loc[submit.index]
new_submit

t  x1        x2   x3
ego_id        u  v                                 
8             0  93   359.600006 NaN  0.000000  0.0
                 143    6.100000 NaN  0.000000  0.0
                 151    0.200000 NaN  1.386294  0.0
              1  24   594.500000 NaN  0.000000  0.0
              5  4    461.500000 NaN  0.000000  0.0
...                          ...  ..       ...  ...
1709396984676 73 23    20.299999 NaN  0.000000  0.0
              74 68    46.700001 NaN  0.000000  0.0
              77 28    43.400002 NaN  0.000000  0.0
              79 38    50.200001 NaN  0.000000  0.0
              81 92     3.600000 NaN  0.693147  0.0

[811033 rows x 4 columns]

In [23]:
new_submit = new_submit[new_submit['x1'].isna()]
new_submit.shape

(810983, 4)

In [30]:
new_submit = new_submit.reset_index().drop_duplicates()
new_submit

,ego_id,u,v,t,x1,x2,x3
0,8,0,93,359.600006,NaN,0.000000,0.0
1,8,0,143,6.100000,NaN,0.000000,0.0
2,8,0,151,0.200000,NaN,1.386294,0.0
3,8,1,24,594.500000,NaN,0.000000,0.0
4,8,5,4,461.500000,NaN,0.000000,0.0
...,...,...,...,...,...,...,...
810978,1709396984676,73,23,20.299999,NaN,0.000000,0.0
810979,1709396984676,74,68,46.700001,NaN,0.000000,0.0
810980,1709396984676,77,28,43.400002,NaN,0.000000,0.0
810981,1709396984676,79,38,50.200001,NaN,0.000000,0.0


In [45]:
submit = submit.reset_index()
submit

,ego_id,u,v,x1
0,8,0,93,0.000000
1,8,0,143,0.000000
2,8,0,151,1.606742
3,8,1,24,0.026496
4,8,5,4,0.159857
...,...,...,...,...
810971,1709396984676,73,23,1.562170
810972,1709396984676,74,68,0.454210
810973,1709396984676,77,28,0.078504
810974,1709396984676,79,38,0.981812


In [56]:
ego_diff = new_submit.groupby('ego_id').count()['u'] - submit.groupby('ego_id').count()['u']
ego_diff

ego_id
8                0
11               0
28               0
31               0
34               0
                ..
1709396984638    0
1709396984653    0
1709396984665    0
1709396984673    0
1709396984676    0
Name: u, Length: 20586, dtype: int64

In [57]:
ego_diff[ego_diff == 1]

ego_id
1434519077054    1
Name: u, dtype: int64

In [59]:
u_diff = new_submit[new_submit['ego_id'] == 1434519077054].groupby('u').count()['v'] - submit[submit['ego_id'] == 1434519077054].groupby('u').count()['v']
u_diff[u_diff == 1]

u
26    1
Name: v, dtype: int64

In [60]:
new_submit['tmp'] = 1
submit['tmp'] = 1

In [61]:
v_diff = new_submit[
    (new_submit['ego_id'] == 1434519077054) & (new_submit['u'] == 26)
].groupby('v').count()['tmp'] - submit[
    (submit['ego_id'] == 1434519077054) & (submit['u'] == 26)
].groupby('v').count()['tmp']

In [62]:
v_diff

v
11     0
17     0
33     0
37     1
270    0
Name: tmp, dtype: int64

In [63]:
new_submit[
    (new_submit['ego_id'] == 1434519077054) & (new_submit['u'] == 26) & (new_submit['v'] == 37)
]

,ego_id,u,v,t,x1,x2,x3,tmp
678481,1434519077054,26,37,20.1,NaN,0.0,0.0,1
678482,1434519077054,26,37,NaN,NaN,0.0,0.0,1


In [64]:
submit[
    (submit['ego_id'] == 1434519077054) & (submit['u'] == 26) & (submit['v'] == 37)
]

,ego_id,u,v,x1,tmp
678481,1434519077054,26,37,1.305834,1


In [67]:
new_submit = new_submit.drop(678481)
new_submit.shape

(810976, 8)

In [72]:
submit = submit.sort_values(["ego_id", "u", "v"])
new_submit = new_submit.sort_values(["ego_id", "u", "v"])
new_submit.index = submit.index

In [84]:
new_submit[[
    'ego_id', 'u', 'v', 't', 'x1', 'x2', 'x3'
]].to_csv('data/new_submit.csv', index=False)